Ici on récupère les données via PISTE l'API permettant d'avoir accès aux données de Légifrance, le code de connexion OAuth est donné sur le Gitlab de Piste à cette [adresse](https://gitlab.com/piste_lab/oauth_connectors/-/blob/master/Python/Oauth2ClientCredentialsTest.py?ref_type=heads). Il est tout de même nécessaire d'installer les modules du code suivant.

In [2]:
%pip install python-dotenv
%pip install requests-oauthlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Pour pour envoyer des requêtes à l'API nous devons d'abord obtenir un token, après s'être inscrit sur le [site de PISTE](https://piste.gouv.fr/). Ce token est obtenu en envoyant une requête contenant mon identifiant client et mon code de client au site d'autorisation. Nous avons ensuite une autorisation d'une heure avec ce token pour exploiter l'API de Légifrance. 

In [3]:
TOKEN_URL = "https://oauth.piste.gouv.fr/api/oauth/token"

import os
from dotenv import load_dotenv
import requests
from requests_oauthlib import OAuth2Session

# Cette partie sert à récupérer les ID de connexions qui sont dans un fichier .env que l'on doit créer à chaque fois que l'on copie le repository github, cela permet de les conserver secret
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# Requette d'un token pour avoir accès à l'API 
res = requests.post(
  TOKEN_URL,
  data={
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "openid"
  }
)

# Récupération du token 
token = res.json()
client = OAuth2Session(client_id, token=token)

Maintenant en utilisant le lien d'exploitation de l'API auquel on rajoute l'endpoints qui permet d'accéder à ce que l'on veut faire nous récupérons les données . La liste des endpoints pour l'API de Légifrance est disponible [ici](https://piste.gouv.fr/index.php?option=com_apiportal&view=apitester&usage=api&apitab=tests&apiName=L%C3%A9gifrance&apiId=7e5a0e1d-ffcc-40be-a405-a1a5c1afe950&managerId=3&type=rest&apiVersion=2.4.2&Itemid=179&swaggerVersion=2.0&lang=fr).

In [4]:
API_HOST = "https://api.piste.gouv.fr/dila/legifrance/lf-engine-app"

In [ ]:
client.post(
    API_HOST+"/search", # On ajoute ici l'endpoint souhaité
    json = {
  "recherche": {
    "filtres": [
      {
        "dates": {
          "start": "1996-01-01",
          "end": "2022-08-31"
        },
        "facette": "DATE_SIGNATURE"
      }
    ],
    "fromAdvancedRecherche": False,
    "champs": [
      {
        "criteres": [
          {
            "valeur": "crimes",
            "operateur": "OU",
            "criteres": [
              {
                "valeur": "délits",
                "operateur": "OU",
                "typeRecherche": "UN_DES_MOTS"
              },
              {
                "valeur": "délinquance",
                "operateur": "OU",
                "typeRecherche": "UN_DES_MOTS"
              }
            ],
            "typeRecherche": "UN_DES_MOTS"
          }
        ],
        "typeChamp": "ALL"
      }
    ],
    "pageSize": 100,
    "pageNumber": 1,
    "operateur": "ET",
    "typePagination": "DEFAUT"
  },
  "fond": "LEGI",
  "sort": "SIGNATURE_DATE_DESC",
  "resultats": {
    "champsRetour": ["title", "date_signature"]
  }
}
).json()

{'timestamp': 1733493185210,
 'error': 400,
 'status': 'Bad Request',
 'message': 'Le format de la requête est invalide. Veuillez vérifier le formatage de votre requête',
 'errorCode': None,
 'exceptionDetails': None}